<a href="https://colab.research.google.com/github/ClaudiaMarano/Anomaly-Detection-and-Prediction/blob/main/PhysicalDenseNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task di anomaly prediction per la parte physical tramite modello basato su Dense NN

Preprocessing

In [12]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

def dataset_load_and_segmentation(path_norm, att_paths=[]):
    """
    Carica il dataset normale e opzionalmente i dataset di attacco, segmentandoli in finestre temporali.

    Args:
        path_norm (str): Percorso del file CSV normale.
        att_paths (list): Lista dei percorsi dei file CSV di attacco.

    Returns:
        list: Segmenti uniformi estratti dal dataset normale e opzionalmente dai dataset di attacco.
    """
    try:
        # Carico il CSV normale con diversa codifica, gestendo file con separatori differenti
        try:
            df_norm = pd.read_csv(path_norm, sep=',', encoding='utf-8')
        except Exception:
            df_norm = pd.read_csv(path_norm, sep='\t', engine='python', encoding='utf-16')

        # Rimuovo eventuali spazi nei nomi delle colonne
        df_norm.columns = df_norm.columns.str.strip()

        # Verifico la presenza della colonna Time
        if 'Time' not in df_norm.columns:
            raise ValueError("La colonna 'Time' non è presente nel file normale.")

        # Rimuovo la colonna "Label"
        if 'Label' in df_norm.columns:
            df_norm = df_norm.drop(columns=["Label"])

        # Converto la colonna Time in datetime
        df_norm['Time'] = pd.to_datetime(df_norm['Time'], dayfirst=True)

        # Ordino per timestamp nel caso non siano già ordinati
        df_norm = df_norm.sort_values(by='Time')

        # Definisco la durata della finestra in un minuto
        window_duration = pd.Timedelta(minutes=1)

        # Lista per i segmenti
        segments = []
        start_time = df_norm['Time'].iloc[0]

        while start_time < df_norm['Time'].iloc[-1]:
            end_time = start_time + window_duration
            segment = df_norm[(df_norm['Time'] >= start_time) & (df_norm['Time'] < end_time)]
            if len(segment) > 0:
                segments.append(segment.drop(columns=['Time']).values)
            start_time = end_time

        # Mantengo solo i segmenti con lunghezza pari a 60
        uniform_segments = [segment for segment in segments if len(segment) == 60]

        # Aggiungo segmenti dai file di attacco se specificati
        if att_paths:
            for path in att_paths:
                segments_from_att = load_from_att_file(path)
                uniform_segments.extend(segments_from_att)

        return uniform_segments
    except Exception as e:
        print(f"Errore durante il caricamento e la segmentazione del dataset: {e}")
        return []

def load_from_att_file(path):
    """
    Carica segmenti uniformi da un file di attacco.

    Args:
        path (str): Percorso del file CSV di attacco.

    Returns:
        list: Segmenti uniformi estratti dal file di attacco.
    """
    try:
        # Carico il CSV di attacco con diversa codifica, gestendo file con separatori differenti
        try:
            df_att = pd.read_csv(path, sep=',', encoding='utf-8')
        except Exception:
            df_att = pd.read_csv(path, sep='\t', engine='python', encoding='utf-16')

        # Rimuovo eventuali spazi nei nomi delle colonne
        df_att.columns = df_att.columns.str.strip()

        # Verifico la presenza della colonna Time
        if 'Time' not in df_att.columns:
            raise ValueError(f"La colonna 'Time' non è presente nel file {path}.")

        # Converto la colonna Time in datetime
        df_att['Time'] = pd.to_datetime(df_att['Time'], dayfirst=True)

        # Ordino per timestamp nel caso non siano già ordinati
        df_att = df_att.sort_values(by='Time')

        # Definisco la durata della finestra in un minuto
        window_duration = pd.Timedelta(minutes=1)

        # Lista per i segmenti
        segments = []
        start_time = df_att['Time'].iloc[0]

        while start_time < df_att['Time'].iloc[-1]:
            end_time = start_time + window_duration
            segment = df_att[(df_att['Time'] >= start_time) & (df_att['Time'] < end_time)]
            if len(segment) > 0:
                segments.append(segment.drop(columns=['Time']).values)
            start_time = end_time

        # Mantengo solo i segmenti con lunghezza pari a 60 e senza anomalie
        uniform_segments = [
            segment for segment in segments
            if len(segment) == 60 and all(row[-1] == 'normal' for row in segment)
        ]

        # Rimuovo l'ultima colonna da ogni segmento
        clean_segments = [segment[:, :-1] for segment in uniform_segments]

        return clean_segments
    except Exception as e:
        print(f"Errore durante il caricamento del file di attacco: {e}")
        return []

def preprocessing(segments):
    """
    Normalizza i segmenti forniti e restituisce i segmenti normalizzati e lo scaler.

    Args:
        segments (list): Lista di segmenti da normalizzare.

    Returns:
        tuple: Segmenti normalizzati e scaler utilizzato.
    """
    try:
        # Unisco tutti i segmenti in un unico array per la normalizzazione
        segments_array = np.vstack(segments)

        # Normalizzo i dati
        scaler = StandardScaler()
        segments_scaled = scaler.fit_transform(segments_array)

        # Divido di nuovo i segmenti normalizzati
        segments_scaled_split = np.array_split(segments_scaled, len(segments))

        return segments_scaled_split, scaler
    except Exception as e:
        print(f"Errore durante il preprocessing dei segmenti: {e}")
        return [], None

def split_dataset(segments, test_size=0.2, val_size=0.1):
    """
    Divide i segmenti in set di training, validation e test.

    Args:
        segments (list): Lista dei segmenti da dividere.
        test_size (float): Proporzione del dataset da assegnare al test set.
        val_size (float): Proporzione del training set da assegnare al validation set.

    Returns:
        tuple: Training, validation e test set.
    """
    try:
        train_val_segments, test_segments = train_test_split(segments, test_size=test_size, random_state=42)
        train_segments, val_segments = train_test_split(train_val_segments, test_size=val_size, random_state=42)

        print(f"Training set: {len(train_segments)} segmenti")
        print(f"Validation set: {len(val_segments)} segmenti")
        print(f"Test set: {len(test_segments)} segmenti")

        return train_segments, val_segments, test_segments
    except Exception as e:
        print(f"Errore durante la suddivisione del dataset: {e}")
        return [], [], []

# Percorsi dei file del dataset Physical
path_norm = "./physical_dataset/phy_norm.csv"
att_paths = [
    "./physical_dataset/phy_att_1.csv",
    "./physical_dataset/phy_att_2.csv",
    "./physical_dataset/phy_att_3.csv",
    "./physical_dataset/phy_att_4.csv"
]

# Caricamento e segmentazione del dataset
segments = dataset_load_and_segmentation(path_norm, att_paths=att_paths)

# Preprocessing dei segmenti
segments_scaled, scaler = preprocessing(segments)

if segments_scaled:
    print("Preprocessing completato con successo.")

    # Suddivisione del dataset
    train_segments, val_segments, test_segments = split_dataset(segments_scaled)
else:
    print("Errore durante il preprocessing dei segmenti.")


Preprocessing completato con successo.
Training set: 76 segmenti
Validation set: 9 segmenti
Test set: 22 segmenti


Implementazine del modello

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split
import numpy as np

# Funzione per preparare i dati per il modello DNN
def prepare_data_for_model(segments):
    try:
        X = np.array([segment[:, :-1].flatten() for segment in segments])  # Flatten delle feature
        y = np.array([segment[0, -1] for segment in segments])  # Target dalla prima riga di ogni segmento

        # Verifica che la label contenga sia anomalie (1) che normalità (0)
        unique_labels = np.unique(y)
        if len(unique_labels) < 2:
            raise ValueError("La colonna 'label_n' non contiene sia anomalie che normalità. Verifica i dati.")

        return X, y
    except Exception as e:
        print(f"Errore durante la preparazione dei dati per il modello: {e}")
        return None, None

# Funzione per definire e addestrare il modello DNN
def train_dnn(X_train, y_train, X_val, y_val):
    try:
        input_dim = X_train.shape[1]  # Calcolo della dimensione dell'input

        # Definizione del modello
        model = Sequential([
            Dense(128, activation='relu', input_dim=input_dim),
            Dropout(0.3),
            Dense(64, activation='relu'),
            Dropout(0.3),
            Dense(1, activation='sigmoid')
        ])

        # Compilazione del modello
        model.compile(optimizer=Adam(learning_rate=0.001),
                      loss='binary_crossentropy',
                      metrics=['accuracy'])

        # Early stopping per evitare overfitting
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

        # Addestramento del modello
        history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                            epochs=50, batch_size=32, callbacks=[early_stopping])

        print("Modello addestrato con successo.")
        return model, history
    except Exception as e:
        print(f"Errore durante l'addestramento del modello: {e}")
        return None, None

# Funzione per valutare il modello
def evaluate_model(model, X_test, y_test):
    try:
        y_pred = (model.predict(X_test) > 0.5).astype(int)
        print("\nConfusion Matrix:")
        print(confusion_matrix(y_test, y_pred))
        print("\nClassification Report:")
        print(classification_report(y_test, y_pred))
        if len(np.unique(y_test)) > 1:
            auc = roc_auc_score(y_test, y_pred)
            print(f"\nROC-AUC Score: {auc:.4f}")
        else:
            print("\nROC-AUC Score non calcolabile: una sola classe presente in y_test.")
    except Exception as e:
        print(f"Errore durante la valutazione del modello: {e}")

# Suddivisione bilanciata dei dati in train, validation e test
def split_data_balanced(X, y, test_size=0.2, val_size=0.1):
    try:
        X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=test_size, stratify=y, random_state=42)
        X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=val_size / (1 - test_size), stratify=y_train_val, random_state=42)
        return X_train, X_val, X_test, y_train, y_val, y_test
    except Exception as e:
        print(f"Errore durante la suddivisione bilanciata dei dati: {e}")
        return None, None, None, None, None, None

# Verifica presenza anomalie nel dataset
def verify_anomalies_presence(segments):
    try:
        labels = [segment[0, -1] for segment in segments]
        unique_labels = np.unique(labels)
        if len(unique_labels) < 2:
            raise ValueError("Il dataset non contiene sia anomalie che normalità. Verifica il preprocessing.")
        print("Verifica completata: Il dataset contiene sia anomalie che normalità.")
    except Exception as e:
        print(f"Errore durante la verifica delle anomalie: {e}")

# Preparazione dei dati per il modello
try:
    verify_anomalies_presence(train_segments + val_segments + test_segments)

    X, y = prepare_data_for_model(train_segments + val_segments + test_segments)

    # Verifica che i dati siano stati preparati correttamente
    if X is not None and y is not None:
        # Suddivisione dei dati in train, validation e test set
        X_train, X_val, X_test, y_train, y_val, y_test = split_data_balanced(X, y)

        if X_train is not None and X_val is not None and X_test is not None:
            # Addestramento del modello
            model, history = train_dnn(X_train, y_train, X_val, y_val)

            # Valutazione del modello
            if model is not None:
                evaluate_model(model, X_test, y_test)
        else:
            print("Errore nella suddivisione dei dati in train, validation e test set.")
    else:
        print("Errore nella preparazione dei dati per il modello.")
except Exception as e:
    print(f"Errore generale durante la pipeline: {e}")


Errore durante la verifica delle anomalie: Il dataset non contiene sia anomalie che normalità. Verifica il preprocessing.
Errore durante la preparazione dei dati per il modello: La colonna 'label_n' non contiene sia anomalie che normalità. Verifica i dati.
Errore nella preparazione dei dati per il modello.


In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

def dataset_load_and_segmentation(path_norm, att_paths=[]):
    try:
        # Carico il CSV normale con diversa codifica, gestendo file con separatori differenti
        try:
            df_norm = pd.read_csv(path_norm, sep=',', encoding='utf-8')
        except Exception:
            df_norm = pd.read_csv(path_norm, sep='\t', engine='python', encoding='utf-16')

        # Rimuovo eventuali spazi nei nomi delle colonne
        df_norm.columns = df_norm.columns.str.strip()

        # Verifico la presenza della colonna Time
        if 'Time' not in df_norm.columns:
            raise ValueError("La colonna 'Time' non è presente nel file normale.")

        # Rimuovo la colonna "Label"
        if 'Label' in df_norm.columns:
            df_norm = df_norm.drop(columns=["Label"])

        # Converto la colonna Time in datetime
        df_norm['Time'] = pd.to_datetime(df_norm['Time'], dayfirst=True)

        # Ordino per timestamp nel caso non siano già ordinati
        df_norm = df_norm.sort_values(by='Time')

        # Filtro solo valori validi (0 e 1) in label_n
        if 'label_n' in df_norm.columns:
            unique_labels = df_norm['label_n'].unique()
            print(f"Valori unici trovati in label_n (normale): {unique_labels}")
            df_norm = df_norm[df_norm['label_n'].isin([0, 1])]

        # Definisco la durata della finestra in un minuto
        window_duration = pd.Timedelta(minutes=1)

        # Lista per i segmenti
        segments = []
        start_time = df_norm['Time'].iloc[0]

        while start_time < df_norm['Time'].iloc[-1]:
            end_time = start_time + window_duration
            segment = df_norm[(df_norm['Time'] >= start_time) & (df_norm['Time'] < end_time)]
            if len(segment) > 0:
                segments.append(segment.drop(columns=['Time']).values)
            start_time = end_time

        # Mantengo solo i segmenti con lunghezza pari a 60
        uniform_segments = [segment for segment in segments if len(segment) == 60]

        # Aggiungo segmenti dai file di attacco se specificati
        if att_paths:
            for path in att_paths:
                segments_from_att = load_from_att_file(path, df_norm.columns)
                uniform_segments.extend(segments_from_att)

        return uniform_segments
    except Exception as e:
        print(f"Errore durante il caricamento e la segmentazione del dataset: {e}")
        return []

def load_from_att_file(path, reference_columns):
    try:
        # Carico il CSV di attacco con diversa codifica, gestendo file con separatori differenti
        try:
            df_att = pd.read_csv(path, sep=',', encoding='utf-8')
        except Exception:
            df_att = pd.read_csv(path, sep='\t', engine='python', encoding='utf-16')

        # Rimuovo eventuali spazi nei nomi delle colonne
        df_att.columns = df_att.columns.str.strip()

        # Verifico la presenza della colonna Time
        if 'Time' not in df_att.columns:
            raise ValueError(f"La colonna 'Time' non è presente nel file {path}.")

        # Converto la colonna Time in datetime
        df_att['Time'] = pd.to_datetime(df_att['Time'], dayfirst=True)

        # Ordino per timestamp nel caso non siano già ordinati
        df_att = df_att.sort_values(by='Time')

        # Filtro solo valori validi (0 e 1) in label_n
        if 'label_n' in df_att.columns:
            unique_labels = df_att['label_n'].unique()
            print(f"Valori unici trovati in label_n (attacco): {unique_labels}")
            df_att = df_att[df_att['label_n'].isin([0, 1])]

        # Aggiusto le colonne mancanti rispetto al dataset di riferimento
        for col in reference_columns:
            if col not in df_att.columns:
                df_att[col] = 0
        df_att = df_att[reference_columns]

        # Definisco la durata della finestra in un minuto
        window_duration = pd.Timedelta(minutes=1)

        # Lista per i segmenti
        segments = []
        start_time = df_att['Time'].iloc[0]

        while start_time < df_att['Time'].iloc[-1]:
            end_time = start_time + window_duration
            segment = df_att[(df_att['Time'] >= start_time) & (df_att['Time'] < end_time)]
            if len(segment) > 0:
                segments.append(segment.drop(columns=['Time']).values)
            start_time = end_time

        # Mantengo solo i segmenti con lunghezza pari a 60
        uniform_segments = [segment for segment in segments if len(segment) == 60]

        return uniform_segments
    except Exception as e:
        print(f"Errore durante il caricamento del file di attacco: {e}")
        return []

def preprocessing(segments):
    try:
        # Separare label_n dalle feature per evitare di normalizzarla
        features_segments = [segment[:, :-1] for segment in segments]
        labels = [segment[0, -1] for segment in segments]

        # Unisco tutti i segmenti in un unico array per la normalizzazione
        segments_array = np.vstack(features_segments)

        # Normalizzo i dati
        scaler = StandardScaler()
        segments_scaled = scaler.fit_transform(segments_array)

        # Divido di nuovo i segmenti normalizzati
        features_scaled_split = np.array_split(segments_scaled, len(segments))

        # Ricostruisco i segmenti includendo le label originali
        segments_scaled_split = [
            np.hstack((features_scaled, np.full((features_scaled.shape[0], 1), label)))
            for features_scaled, label in zip(features_scaled_split, labels)
        ]

        return segments_scaled_split, scaler
    except Exception as e:
        print(f"Errore durante il preprocessing dei segmenti: {e}")
        return [], None

def verify_anomalies_presence(segments):
    try:
        labels = [segment[0, -1] for segment in segments]
        unique_labels = np.unique(labels)
        print(f"Valori unici nella colonna 'label_n' dopo preprocessing: {unique_labels}")
        if len(unique_labels) < 2:
            raise ValueError("Il dataset non contiene sia anomalie che normalità. Verifica il preprocessing.")
        print("Verifica completata: Il dataset contiene sia anomalie che normalità.")
    except Exception as e:
        print(f"Errore durante la verifica delle anomalie: {e}")

def prepare_data_for_model(segments):
    try:
        X = np.array([segment[:, :-1].flatten() for segment in segments])  # Flatten delle feature
        y = np.array([segment[0, -1] for segment in segments], dtype=int)  # Target dalla prima riga di ogni segmento

        # Verifica che la label contenga sia anomalie (1) che normalità (0)
        unique_labels = np.unique(y)
        print(f"Valori unici in 'y' prima del modello: {unique_labels}")
        if len(unique_labels) < 2:
            raise ValueError("La colonna 'label_n' non contiene sia anomalie che normalità. Verifica i dati.")

        return X, y
    except Exception as e:
        print(f"Errore durante la preparazione dei dati per il modello: {e}")
        return None, None

def split_data_balanced(X, y, test_size=0.2, val_size=0.1):
    try:
        X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=test_size, stratify=y, random_state=42)
        X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=val_size / (1 - test_size), stratify=y_train_val, random_state=42)
        return X_train, X_val, X_test, y_train, y_val, y_test
    except Exception as e:
        print(f"Errore durante la suddivisione bilanciata dei dati: {e}")
        return None, None, None, None, None, None

def train_dnn(X_train, y_train, X_val, y_val):
    try:
        input_dim = X_train.shape[1]  # Calcolo della dimensione dell'input

        # Definizione del modello
        model = Sequential([
            Dense(128, activation='relu', input_dim=input_dim),
            Dropout(0.3),
            Dense(64, activation='relu'),
            Dropout(0.3),
            Dense(1, activation='sigmoid')
        ])

        # Compilazione del modello
        model.compile(optimizer=Adam(learning_rate=0.001),
                      loss='binary_crossentropy',
                      metrics=['accuracy'])

        # Early stopping per evitare overfitting
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

        # Addestramento del modello
        history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                            epochs=50, batch_size=32, callbacks=[early_stopping])

        print("Modello addestrato con successo.")
        return model, history
    except Exception as e:
        print(f"Errore durante l'addestramento del modello: {e}")
        return None, None

def evaluate_model(model, X_test, y_test):
    try:
        y_pred = (model.predict(X_test) > 0.5).astype(int)
        print("\nConfusion Matrix:")
        print(confusion_matrix(y_test, y_pred))
        print("\nClassification Report:")
        print(classification_report(y_test, y_pred))
        if len(np.unique(y_test)) > 1:
            auc = roc_auc_score(y_test, y_pred)
            print(f"\nROC-AUC Score: {auc:.4f}")
        else:
            print("\nROC-AUC Score non calcolabile: una sola classe presente in y_test.")
    except Exception as e:
        print(f"Errore durante la valutazione del modello: {e}")

# Pipeline
path_norm = "./physical_dataset/phy_norm.csv"
att_paths = [
    "./physical_dataset/phy_att_1.csv",
    "./physical_dataset/phy_att_2.csv",
    "./physical_dataset/phy_att_3.csv",
    "./physical_dataset/phy_att_4.csv"
]

# Caricamento e segmentazione del dataset
segments = dataset_load_and_segmentation(path_norm, att_paths=att_paths)

# Preprocessing dei segmenti
segments_scaled, scaler = preprocessing(segments)

if segments_scaled:
    print("Preprocessing completato con successo.")

    # Verifica anomalie
    verify_anomalies_presence(segments_scaled)

    # Preparazione dei dati
    X, y = prepare_data_for_model(segments_scaled)

    if X is not None and y is not None:
        # Suddivisione dei dati
        X_train, X_val, X_test, y_train, y_val, y_test = split_data_balanced(X, y)

        if X_train is not None and X_val is not None and X_test is not None:
            # Addestramento del modello
            model, history = train_dnn(X_train, y_train, X_val, y_val)

            # Valutazione del modello
            if model is not None:
                evaluate_model(model, X_test, y_test)
        else:
            print("Errore nella suddivisione dei dati in train, validation e test set.")
    else:
        print("Errore nella preparazione dei dati per il modello.")
else:
    print("Errore durante il preprocessing dei segmenti.")


Preprocessing completato con successo.
Valori unici nella colonna 'label_n' dopo preprocessing: [0. 1.]
Verifica completata: Il dataset contiene sia anomalie che normalità.
Valori unici in 'y' prima del modello: [0 1]
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 73ms/step - accuracy: 0.5581 - loss: 0.8038 - val_accuracy: 0.7778 - val_loss: 0.5580
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8590 - loss: 0.4375 - val_accuracy: 0.7778 - val_loss: 0.7725
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8338 - loss: 0.4116 - val_accuracy: 0.7778 - val_loss: 0.9017
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8445 - loss: 0.5007 - val_accuracy: 0.7778 - val_loss: 0.9274
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8421 - loss: 0.4135 - val_accuracy: 0.7222 - val_loss: 1.0493
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8497 - loss: 0.3951 - val_accuracy: 0.7778 - val_loss: 1.0805
Modello addestrato con successo.
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step

Confusion Matrix:
[[31  0]
 [ 5  0]]

Classification Report:
              precision    recall  f1-score   support

           0       0.86      1.00      0.93        31
           1       

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
